### Python Libraries and Modules for Data Analysis

This notebook utilizes a variety of Python libraries for data analysis and machine learning. Below is a summary of the imported libraries and their roles:


1. **Data Manipulation and Loading**
   - `pandas as pd`: Provides high-performance data structures and data analysis tools.
   - `numpy as np`: Supports large, multi-dimensional arrays and matrices with mathematical functions.
   - `scipy.io`: Contains functions for reading and writing data in MATLAB `.mat` file format.
     - `loadmat`: Loads MATLAB `.mat` files.
   - `h5py`: Reads and writes HDF5 files, including `.mat` files that use the HDF5 format.
   - `glob`: Finds all pathnames matching a specified pattern, useful for file searching.
   - `os`: Provides a way of using operating system-dependent functionality like reading or writing to the file system.
   - `re`: Supports regular expressions for string manipulation.

2. **Data Preprocessing and Feature Engineering**
   - `scipy.signal`: Provides signal processing functions.
     - `hilbert`: Computes the analytic signal using the Hilbert transform.
     - `butter`: Designs a Butterworth filter.
     - `filtfilt`: Applies a forward and backward filter to avoid phase distortion.
     - `detrend`: Removes linear trend from data.

4. **Randomization and Miscellaneous Utilities**
   - `random`: Implements pseudo-random number generators for various distributions.

These libraries and modules facilitate efficient data manipulation, preprocessing, model evaluation, and file handling throughout the analysis.

In [ ]:
import pandas as pd
import numpy as np
import joblib
from scipy.io import loadmat
import scipy.io as sio
import random
from scipy import signal
from scipy.signal import hilbert, butter, filtfilt, detrend
import h5py
import glob
import os
import re


### All relevant data files i.e. the raw MEG .mat files should be uploaded to google drive, imported and mounted into this notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




# Accessing and Inspecting .mat Files from Google Drive

This section demonstrates how to access and inspect `.mat` files stored in Google Drive using the `h5py` library. The script will:

1. Specify the path to the `.mat` file.
2. Open the file using `h5py`.
3. Print out the keys and details of datasets and groups contained in the file.



In [ ]:
# Specify the file path to the .mat file in Google Drive
mat_file_path = '/content/drive/MyDrive/MEG DATA/LSD_DRU_subject_15.mat'

# Open the .mat file in read mode using h5py, and ensure it closes automatically after the block
with h5py.File(mat_file_path, 'r') as f:

    # Print the keys (dataset and group names) present in the .mat file
    print("Keys in the .mat file:", list(f.keys()))

    # Loop through each key in the .mat file to inspect its contents
    for key in f.keys():
        item = f[key]  # Retrieve the item associated with the current key

        # Check if the item is a Group (a collection of datasets or sub-groups)
        if isinstance(item, h5py.Group):
            print(f"Group {key}:")  # Print the name of the group

            # Loop through each key in the group to inspect subgroups or datasets
            for subgroup_key in item.keys():
                subgroup_item = item[subgroup_key]  # Retrieve the item in the subgroup
                # Print details about the subgroup: its shape and data type
                print(f"  Subgroup {subgroup_key}: shape {subgroup_item.shape}, dtype {subgroup_item.dtype}")

        # If the item is a Dataset, print its shape and data type
        elif isinstance(item, h5py.Dataset):
            print(f"Dataset {key}: shape {item.shape}, dtype {item.dtype}")


Keys in the .mat file: ['#refs#', 'X', 'lnfilter']
Group #refs#:
  Subgroup a: shape (2,), dtype uint64
Dataset X: shape (209, 1200, 90), dtype float64
Group lnfilter:
  Subgroup fdesc: shape (4, 1), dtype uint16


### This cell outlines the comprehensive pipeline and steps involved in processing MEG data from a `.mat` file. The workflow includes defining regions of interest (ROIs), applying preprocessing techniques, and calculating the Lempel-Ziv (LZ) complexity for each ROI.

In [ ]:
# Define ROIs and their corresponding regions
roi_regions = {
    'sensorimotor': [1, 2, 17, 18, 57, 58],
    'frontal': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28],
    'temporal': [29, 30, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90],
    'cingulate': [31, 32, 33, 34, 35, 36],
    'limbic': [37, 38, 39, 40, 41, 42],
    'occipital': [43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56],
    'parietal': [59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70],
    'subcortical': [71, 72, 73, 74, 75, 76, 77, 78]
}



def select_fixed_rois(roi_regions, num_samples_per_region=3):

      """
    Selects a fixed number of Regions of Interest (ROIs) from each specified brain region.

    Parameters:
    roi_regions (dict): A dictionary where keys are brain region names, and values are lists of ROIs in those regions.
    num_samples_per_region (int): The number of ROIs to select from each brain region.

    Returns:
    selected_rois (list): A list of selected ROIs.
    roi_names (list): A list of region names corresponding to the selected ROIs.
    """
    selected_rois = []
    roi_names = []
    for region, rois in roi_regions.items():
        selected_rois.extend(rois[:num_samples_per_region])
        roi_names.extend([region] * num_samples_per_region)
    return selected_rois, roi_names

def apply_bandpass_filter(data, low_cutoff, high_cutoff, fs):

  """
    Applies a bandpass filter to the input data.

    Parameters:
    data (np.array): The input signal data to be filtered (2D array with shape [samples, channels]).
    low_cutoff (float): The low frequency cutoff for the bandpass filter.
    high_cutoff (float): The high frequency cutoff for the bandpass filter.
    fs (float): The sampling frequency of the data.

    Returns:
    filtered_data (np.array): The filtered data with the same shape as the input data.
    """
    print("Data shape before filtering:", data.shape)
    nyq = 0.5 * fs
    low = low_cutoff / nyq
    high = high_cutoff / nyq
    b, a = butter(4, [low, high], btype='band')
    filtered_data = np.zeros_like(data)
    for i in range(data.shape[1]):
        filtered_data[:, i] = filtfilt(b, a, data[:, i])
    print("Data shape after filtering:", filtered_data.shape)
    return filtered_data

def Pre(X):

   """
    Preprocesses the input data by detrending and centering it around zero.

    Parameters:
    X (np.array): The input data to preprocess (2D array with shape [epochs, channels]).

    Returns:
    Z (np.array): The preprocessed data with the same shape as the input.
    """
    epochs, ro = X.shape
    Z = np.zeros((epochs, ro))
    for e in range(epochs):
        Z[e, :] = detrend(X[e, :] - np.mean(X[e, :]), axis=0)
    return Z

def cpr(string):

  """
    Calculates the Lempel-Ziv complexity of a binary string.

    Parameters:
    string (str): The input binary string.

    Returns:
    int: The Lempel-Ziv complexity, which is the length of the dictionary built from the string.
    """
    d = {}
    w = ''
    for c in string:
        wc = w + c
        if wc in d:
            w = wc
        else:
            d[wc] = wc
            w = c
    return len(d)

def str_col(X):

   """
    Converts the input signal data into a binary string based on amplitude thresholds.

    Parameters:
    X (np.array): The input signal data (2D array with shape [epochs, channels]).

    Returns:
    str: A binary string representation of the input data.
    """
    epochs, ro = X.shape
    TH = np.zeros(epochs)
    M = np.zeros((epochs, ro))
    for e in range(epochs):
        M[e, :] = np.abs(hilbert(X[e, :]))
        TH[e] = np.mean(M[e, :])

    s = ''
    for e in range(epochs):
        for i in range(ro):
            if M[e, i] > TH[e]:
                s += '1'
            else:
                s += '0'
    return s

def LZc(X):

  """
    Computes the Lempel-Ziv complexity of preprocessed signal data.

    Parameters:
    X (np.array): The input signal data (2D array with shape [epochs, channels]).

    Returns:
    float: The normalized Lempel-Ziv complexity value.
    """
    X = Pre(X)
    SC = str_col(X)
    M = list(SC)
    random.shuffle(M)
    w = ''
    for i in range(len(M)):
        w += M[i]
    return cpr(SC) / float(cpr(w))

def load_mat_file_h5py(mat_file_path):

   """
    Loads data from a .mat file using the h5py library.

    Parameters:
    mat_file_path (str): The file path to the .mat file.

    Returns:
    np.array: The data extracted from the .mat file.
    """

    with h5py.File(mat_file_path, 'r') as f:
        print("Keys in the .mat file:", list(f.keys()))
        data = f['X'][:]
        print("Loaded data shape:", data.shape)
        return data

def process_file(mat_file_path):
  """
    Processes a .mat file by loading data, sampling ROIs, and printing data shapes.

    Parameters:
    mat_file_path (str): The file path to the .mat file.

    Steps:
    - Load data from the .mat file.
    - Select a fixed number of ROIs and their corresponding names.
    - For each selected ROI, extract the relevant data and print its shape.
    """

    # Load the data
    data = load_mat_file_h5py(mat_file_path)

    # Sample fixed ROIs and get their names
    sampled_rois, roi_names = select_fixed_rois(roi_regions, num_samples_per_region=3)

    results = []

    for i, roi in enumerate(sampled_rois):
        # Extract the data for the current ROI
        selected_data = data[:, roi, :]
        print(f"Data shape for ROI {roi}: {selected_data.shape}")

        # Perform downsampling
        decimation_factor = 3
        downsampled_data = selected_data[:, ::decimation_factor]
        print(f"Data shape after downsampling for ROI {roi}: {downsampled_data.shape}")

        # Apply bandpass filter
        filtered_data = apply_bandpass_filter(downsampled_data, low_cutoff=0.1, high_cutoff=30, fs=600)

        # Compute LZ complexity
        lz_complexity = LZc(filtered_data)

        # Append the result
        results.append((roi, roi_names[i], lz_complexity))

    return results

# Example usage for one file
file_path = '/content/drive/MyDrive/MEG DATA/KET_DRU_subject_01.mat'
lz_complexities = process_file(file_path)

# Print results
for roi, roi_name, lz_complexity in lz_complexities:
    print(f'Channel: {roi}, ROI: {roi_name}, LZ Complexity: {lz_complexity}')


Keys in the .mat file: ['#refs#', 'X', 'lnfilter']
Loaded data shape: (277, 1200, 90)
Data shape for ROI 1: (277, 90)
Data shape after downsampling for ROI 1: (277, 30)
Data shape before filtering: (277, 30)
Data shape after filtering: (277, 30)
Data shape for ROI 2: (277, 90)
Data shape after downsampling for ROI 2: (277, 30)
Data shape before filtering: (277, 30)
Data shape after filtering: (277, 30)
Data shape for ROI 17: (277, 90)
Data shape after downsampling for ROI 17: (277, 30)
Data shape before filtering: (277, 30)
Data shape after filtering: (277, 30)
Data shape for ROI 3: (277, 90)
Data shape after downsampling for ROI 3: (277, 30)
Data shape before filtering: (277, 30)
Data shape after filtering: (277, 30)
Data shape for ROI 4: (277, 90)
Data shape after downsampling for ROI 4: (277, 30)
Data shape before filtering: (277, 30)
Data shape after filtering: (277, 30)
Data shape for ROI 5: (277, 90)
Data shape after downsampling for ROI 5: (277, 30)
Data shape before filtering: 

### Calculating Lempel Ziv for each subject file

####This script handles and iterates through all 97 .mat files containing the raw MEG data, performs random sampling of ROIs, and processes each file to compute and average LZ complexity measures. Results are collected for all 3 psychedelics and conditions (drug/placebo), saved to a CSV file, and prepared for further analysis.                                                        NB: The computational load is substantial due to the large number of files and extensive data processing. As a result, execution may take a considerable amount of time. Please plan accordingly.

In [ ]:
# Define ROIs and their corresponding regions
roi_regions = {
    'sensorimotor': [1, 2, 17, 18, 57, 58],
    'frontal': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28],
    'temporal': [29, 30, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90],
    'cingulate': [31, 32, 33, 34, 35, 36],
    'limbic': [37, 38, 39, 40, 41, 42],
    'occipital': [43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56],
    'parietal': [59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70],
    'subcortical': [71, 72, 73, 74, 75, 76, 77, 78]
}

# Function to randomly select channels from each ROI
def select_random_rois(roi_regions, num_samples_per_region=3):
    selected_rois = []
    roi_names = []
    for region, rois in roi_regions.items():
        sampled_rois = random.sample(rois, num_samples_per_region)
        selected_rois.extend(sampled_rois)
        roi_names.extend([region] * num_samples_per_region)
    return selected_rois, roi_names

# Function to apply a bandpass filter to the data
def apply_bandpass_filter(data, low_cutoff, high_cutoff, fs):
    nyq = 0.5 * fs
    low = low_cutoff / nyq
    high = high_cutoff / nyq
    b, a = butter(4, [low, high], btype='band')
    filtered_data = np.zeros_like(data)
    for i in range(data.shape[1]):
        filtered_data[:, i] = filtfilt(b, a, data[:, i])
    return filtered_data

# Function to detrend and preprocess the data
def Pre(X):
    epochs, ro = X.shape
    Z = np.zeros((epochs, ro))
    for e in range(epochs):
        Z[e, :] = detrend(X[e, :] - np.mean(X[e, :]), axis=0)
    return Z

# Function to detrend and preprocess the data
def cpr(string):
    d = {}
    w = ''
    for c in string:
        wc = w + c
        if wc in d:
            w = wc
        else:
            d[wc] = wc
            w = c
    return len(d)

# Function to convert data into a binary string representation
def str_col(X):
    epochs, ro = X.shape
    TH = np.zeros(epochs)
    M = np.zeros((epochs, ro))
    for e in range(epochs):
        M[e, :] = np.abs(hilbert(X[e, :]))
        TH[e] = np.mean(M[e, :])

    s = ''
    for e in range(epochs):
        for i in range(ro):
            if M[e, i] > TH[e]:
                s += '1'
            else:
                s += '0'
    return s



# Function to compute the Lempel-Ziv complexity
def LZc(X):
    X = Pre(X)
    SC = str_col(X)
    M = list(SC)
    random.shuffle(M)
    w = ''
    for i in range(len(M)):
        w += M[i]
    return cpr(SC) / float(cpr(w))

# Function to load .mat files using h5py
def load_mat_file_h5py(mat_file_path):
    try:
        with h5py.File(mat_file_path, 'r') as f:
            print("Keys in the .mat file:", list(f.keys()))
            data = f['X'][:]  # Replace 'X' with the actual key for your data
            return data
    except (OSError, KeyError) as e:
        print(f"Error loading file {mat_file_path}: {e}")
        return None

# Function to process a .mat file, including data filtering, LZ complexity calculation, and averaging results
def process_file(mat_file_path, num_iterations=30):
    # Load the data
    data = load_mat_file_h5py(mat_file_path)
    if data is None:
        return []

    results = []

    for _ in range(num_iterations):
        # Sample random ROIs and get their names (brain region)
        sampled_rois, roi_names = select_random_rois(roi_regions, num_samples_per_region=3)

        for i, roi in enumerate(sampled_rois):
            try:
                # Extract the data for the current ROI
                selected_data = data[:, roi, :]

                # Perform downsampling to 200Hz
                decimation_factor = 3
                downsampled_data = selected_data[:, ::decimation_factor]

                # Apply bandpass filter
                filtered_data = apply_bandpass_filter(downsampled_data, low_cutoff=0.1, high_cutoff=30, fs=600)

                # Compute LZ complexity
                lz_complexity = LZc(filtered_data)

                # Append the result
                results.append((roi, roi_names[i], lz_complexity))
            except Exception as e:
                print(f"Failed to process ROI {roi} in file {mat_file_path} due to error: {e}")

    # Calculate the average LZ complexity for each ROI
    averaged_results = {}
    for roi, roi_name, lz_complexity in results:
        if (roi, roi_name) not in averaged_results:
            averaged_results[(roi, roi_name)] = []
        averaged_results[(roi, roi_name)].append(lz_complexity)

    final_results = []
    for (roi, roi_name), complexities in averaged_results.items():
        avg_complexity = np.mean(complexities)
        final_results.append((roi, roi_name, avg_complexity))

    return final_results

# Function to extract the subject number from the file path
def extract_subject_number(file_path):
    base_name = os.path.basename(file_path)
    match = re.search(r'subject_(\d+)', base_name)
    if match:
        return int(match.group(1))
    return None

# Main function to collect data across different conditions and psychedelics
def collect_data(base_path, psychedelics, conditions, num_iterations=30):
    all_results = []
    error_files = []

    for psychedelic in psychedelics:
        for condition in conditions:
            file_pattern = f"{base_path}/{psychedelic}_{condition}_subject_*.mat"
            file_paths = glob.glob(file_pattern)

            for file_path in file_paths:
                print(f"Processing file: {file_path}")
                try:
                    subject_number = extract_subject_number(file_path)
                    lz_complexities = process_file(file_path, num_iterations)

                    for roi, roi_name, lz_complexity in lz_complexities:
                        all_results.append({
                            'Psychedelic': psychedelic,
                            'Condition': condition,
                            'Subject': subject_number,
                            'Channel': roi,
                            'ROI': roi_name,
                            'LZ Complexity': lz_complexity
                        })
                except Exception as e:
                    print(f"Failed to process file {file_path} due to error: {e}")
                    error_files.append(file_path)

    if error_files:
        print("Files that caused errors:")
        for error_file in error_files:
            print(error_file)

    return pd.DataFrame(all_results)

# Define parameters
base_path = '/content/drive/MyDrive/MEG DATA'
psychedelics = ['LSD', 'KET', 'PSI']
conditions = ['DRU', 'PLA']

# Collect the data into a dataframe
LZ_df = collect_data(base_path, psychedelics, conditions, num_iterations=10)

# Save results to CSV
LZ_df.to_csv('lz_complexities.csv', index=False)

# Print the first few rows of results_df
print(LZ_df.head())


Processing file: /content/drive/MyDrive/MEG DATA/LSD_DRU_subject_15.mat
Keys in the .mat file: ['#refs#', 'X', 'lnfilter']
Processing file: /content/drive/MyDrive/MEG DATA/LSD_DRU_subject_14.mat
Keys in the .mat file: ['#refs#', 'X', 'lnfilter']
Processing file: /content/drive/MyDrive/MEG DATA/LSD_DRU_subject_13.mat
Keys in the .mat file: ['#refs#', 'X', 'lnfilter']
Processing file: /content/drive/MyDrive/MEG DATA/LSD_DRU_subject_12.mat
Keys in the .mat file: ['#refs#', 'X', 'lnfilter']
Processing file: /content/drive/MyDrive/MEG DATA/LSD_DRU_subject_11.mat
Keys in the .mat file: ['#refs#', 'X', 'lnfilter']
Processing file: /content/drive/MyDrive/MEG DATA/LSD_DRU_subject_09.mat
Keys in the .mat file: ['#refs#', 'X', 'lnfilter']
Processing file: /content/drive/MyDrive/MEG DATA/LSD_DRU_subject_10.mat
Keys in the .mat file: ['#refs#', 'X', 'lnfilter']
Processing file: /content/drive/MyDrive/MEG DATA/LSD_DRU_subject_08.mat
Keys in the .mat file: ['#refs#', 'X', 'lnfilter']
Processing file:

### ONCE RESULTS HAVE BEEN SAVED TO CSV, PLEASE PROCEED TO THE 'PRIMARY ANALYSIS & ML PIPELINE JUPYTER' NOTEBOOK.  
